In [14]:
# --- TASK 3: AI Music Generation (Colab Ready) ---
!pip install music21 tensorflow

import music21 as m21
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import random

# ---------------------------
# STEP 1: Create Dummy Notes Dataset
# ---------------------------
# For demo: Instead of large dataset, we'll use a few notes
notes = ["C4", "D4", "E4", "F4", "G4", "A4", "B4"]
sequence_length = 5

# Generate random sequences of notes
all_sequences = []
output_notes = []
for i in range(100):
    seq = random.choices(notes, k=sequence_length)
    all_sequences.append(seq)
    output_notes.append(random.choice(notes))

# Map notes to integers
note_to_int = {n: i for i, n in enumerate(notes)}
int_to_note = {i: n for n, i in note_to_int.items()}

X = np.array([[note_to_int[n] for n in seq] for seq in all_sequences])
y = np.array([note_to_int[n] for n in output_notes])

# One-hot encode target
y = tf.keras.utils.to_categorical(y, num_classes=len(notes))

print("Training sequences:", X.shape, y.shape)

# ---------------------------
# STEP 2: Build LSTM Model
# ---------------------------
model = models.Sequential([
    layers.Embedding(len(notes), 50, input_length=sequence_length),
    layers.LSTM(128, return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(notes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# ---------------------------
# STEP 3: Train Model
# ---------------------------
model.fit(X, y, epochs=20, batch_size=16, verbose=1)

# ---------------------------
# STEP 4: Generate New Music
# ---------------------------
def generate_notes(seed_seq, num_notes=20):
    generated = seed_seq[:]
    for _ in range(num_notes):
        x_input = np.array([[note_to_int[n] for n in generated[-sequence_length:]]])
        pred = model.predict(x_input, verbose=0)
        next_note = int_to_note[np.argmax(pred)]
        generated.append(next_note)
    return generated

# Pick a random seed
seed = random.choice(all_sequences)
generated_notes = generate_notes(seed, num_notes=20)
print("Generated Notes:", generated_notes)

# ---------------------------
# STEP 5: Save as MIDI
# ---------------------------
stream = m21.stream.Stream()
for n in generated_notes:
    stream.append(m21.note.Note(n))

stream.write("midi", fp="generated_music.mid")
print("✅ Music saved as generated_music.mid")


Training sequences: (100, 5) (100, 7)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0704 - loss: 1.9452   
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2120 - loss: 1.9347
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2016 - loss: 1.9224
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2118 - loss: 1.9042
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2252 - loss: 1.8800
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2030 - loss: 1.8805
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2510 - loss: 1.8394
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2027 - loss: 1.8478
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2222 - loss: 1.8066
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2791 - loss: 1.7850
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3680 - loss: 1.7390 
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2887 - loss: 1.751